# 中间件flask-restful

前面我们手工写api,绑定view,然后返回一个json,这种方式相当原始,

我们可以用[`flask-restful`](http://flask-restful.readthedocs.org/en/latest/index.html)来创建api,它可以直接pip安装

Flask-RESTful 提供的最主要的基础就是资源(resources)。资源(Resources)是构建在 Flask 可拔插视图 之上，只要在你的资源(resource)上定义方法就能够容易地访问多个 HTTP 方法。一个待办事项应用程序的基本的 CRUD 资源看起来像这样:

In [2]:
%%writefile codes/c4/app.py
# --*-- coding:utf-8 --*--
from flask import Flask,request,jsonify
from flask_restful import Api,Resource
from functools import reduce
app = Flask(__name__)
api = Api(app)
todos = {"+":lambda *x:sum(x),
        "-":lambda *x:reduce(lambda i,j:i-j,x),
        "*":lambda *x:sum(x),
        "/":lambda *x:reduce(lambda i,j:i/j,x)
        }

class TodoSimple(Resource):
         
    def get(self,todo_id):
        return jsonify({todo_id:todos.get(todo_id,lambda x,y: "undefined")(10,5)})
         
    def put(self, todo_id):
        todos[todo_id] = eval(str(request.form['data']))
        return {todo_id: str(request.form['data'])}

api.add_resource(TodoSimple, '/<string:todo_id>')

if __name__ == '__main__':
    app.run(debug=True)



Overwriting codes/c4/app.py


In [1]:
from requests import put, get,delete,post

In [2]:
get('http://localhost:5000/+').json()

{u'+': 15}

In [3]:
get('http://localhost:5000/-').json()

{u'-': 5}

In [4]:
put('http://localhost:5000/%', data={'data': 'lambda *x: reduce(lambda i,j:i%j,x)'}).json()

{u'%': u'lambda *x: reduce(lambda i,j:i%j,x)'}

In [5]:
get('http://localhost:5000/%').json()

{u'%': 0}

## 多url共用api

In [7]:
%%writefile codes/c4/app2.py
# --*-- coding:utf-8 --*--
from flask import Flask,request,jsonify
from flask_restful import Api,Resource
from functools import reduce
app = Flask(__name__)
api = Api(app)
todos = {"+":lambda *x:sum(x),
        "-":lambda *x:reduce(lambda i,j:i-j,x),
        "*":lambda *x:sum(x),
        "/":lambda *x:reduce(lambda i,j:i/j,x)
        }

class TodoSimple(Resource):
         
    def get(self,todo_id):
        return jsonify({todo_id:todos.get(todo_id,lambda x,y: "undefined")(10,5)})
         
    def put(self, todo_id):
        todos[todo_id] = eval(str(request.form['data']))
        return {todo_id: str(request.form['data'])}

api.add_resource(TodoSimple,
                 '/<string:todo_id>',
                '/api/<string:todo_id>')

if __name__ == '__main__':
    app.run(debug=True)



Overwriting codes/c4/app2.py


In [8]:
get('http://localhost:5000/api/-').json()

{u'-': 5}

##  解析参数(post,put)

尽管 Flask 能够简单地访问请求数据(比如查询字符串或者 POST 表单编码的数据)，验证表单数据仍然很痛苦。Flask-RESTful 内置了支持验证请求数据，它使用了一个类似 argparse 的库。

需要注意地是与 argparse 模块不同，reqparse.RequestParser.parse_args() 返回一个 Python 字典而不是一个自定义的数据结构。

使用 reqparse 模块同样可以自由地提供聪明的错误信息。如果参数没有通过验证，Flask-RESTful 将会以一个 400 错误请求以及高亮的错误信息回应。

## 数据格式化

默认情况下，在你的返回迭代中所有字段将会原样呈现。尽管当你刚刚处理 Python 数据结构的时候，觉得这是一个伟大的工作，但是当实际处理它们的时候，会觉得十分沮丧和枯燥。为了解决这个问题，Flask-RESTful 提供了 fields 模块和 marshal_with() 装饰器。类似 Django ORM 和 WTForm，你可以使用 fields 模块来在你的响应中格式化结构。

In [9]:
%%writefile codes/c4/app3.py
# --*-- coding:utf-8 --*--
from flask import Flask
from flask_restful import reqparse, abort, Api, Resource

app = Flask(__name__)
api = Api(app)

TODOS = {
    'todo1': {'task': 'build an API'},
    'todo2': {'task': '?????'},
    'todo3': {'task': 'profit!'},
}


def abort_if_todo_doesnt_exist(todo_id):
    if todo_id not in TODOS:
        abort(404, message="Todo {} doesn't exist".format(todo_id))

parser = reqparse.RequestParser()
parser.add_argument('task', type=str)


# Todo
#   show a single todo item and lets you delete them
class Todo(Resource):
    def get(self, todo_id):
        abort_if_todo_doesnt_exist(todo_id)
        return TODOS[todo_id]

    def delete(self, todo_id):
        abort_if_todo_doesnt_exist(todo_id)
        del TODOS[todo_id]
        return '', 204

    def put(self, todo_id):
        args = parser.parse_args()
        task = {'task': args['task']}
        TODOS[todo_id] = task
        return task, 201


# TodoList
#   shows a list of all todos, and lets you POST to add new tasks
class TodoList(Resource):
    def get(self):
        return TODOS

    def post(self):
        args = parser.parse_args()
        todo_id = int(max(TODOS.keys()).lstrip('todo')) + 1
        todo_id = 'todo%i' % todo_id
        TODOS[todo_id] = {'task': args['task']}
        return TODOS[todo_id], 201

##
## Actually setup the Api resource routing here
##
api.add_resource(TodoList, '/todos')
api.add_resource(Todo, '/todos/<todo_id>')


if __name__ == '__main__':
    app.run(debug=True)

Writing codes/c4/app3.py


In [10]:
get('http://localhost:5000/todos').json()#查看全部

{u'todo1': {u'task': u'build an API'},
 u'todo2': {u'task': u'?????'},
 u'todo3': {u'task': u'profit!'}}

In [11]:
get('http://localhost:5000/todos/todo1').json()#查看单独

{u'task': u'build an API'}

In [12]:
delete('http://localhost:5000/todos/todo3')#删除

<Response [204]>

In [13]:
get('http://localhost:5000/todos').json()

{u'todo1': {u'task': u'build an API'}, u'todo2': {u'task': u'?????'}}

In [14]:
post('http://localhost:5000/todos',data={"task":"something new"}).json()#新增

{u'task': u'something new'}

In [15]:
get('http://localhost:5000/todos').json()

{u'todo1': {u'task': u'build an API'},
 u'todo2': {u'task': u'?????'},
 u'todo3': {u'task': u'something new'}}

In [16]:
put('http://localhost:5000/todos/todo3',data={'task':'something different'})#更新

<Response [201]>

In [17]:
get('http://localhost:5000/todos').json()

{u'todo1': {u'task': u'build an API'},
 u'todo2': {u'task': u'?????'},
 u'todo3': {u'task': u'something different'}}